In [ ]:
##indexing##
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import ChatOllama
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
loader=WebBaseLoader(
    web_path=("https://lilianweng.github.io/posts/2023-06-23-agent/"),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        ),
    )
)

docs=loader.load()

#split
text_splitter=RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=300,chunk_overlap=50)
splits=text_splitter.split_documents(docs) #->list

#embed
embedding_model=HuggingFaceEmbeddings(model_name="BAAI/bge-small-zh-v1.5")
vectorstore=Chroma.from_documents(documents=splits,embedding=embedding_model)
retriever=vectorstore.as_retriever()

#ollama llm
llm=ChatOllama(model="llama3.2:3b-instruct-q8_0",temperature=0)

c:\Users\user\anaconda3\envs\torch-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
# HyDE

template="""
Please write a scientific paper passage to answer the question
Question: {question}
Passage:
"""

prompt_hyde=ChatPromptTemplate.from_template(template)
generate_docs_for_retrieval=(
    prompt_hyde
    | llm
    | StrOutputParser()
)
question = "What is task decomposition for LLM agents?"

# Retrieve

retrieval_chain = generate_docs_for_retrieval | retriever

retrieval_chain.invoke({"question":question})


[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Component One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.\nTree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a

In [3]:
from operator import itemgetter
# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""
prompt=ChatPromptTemplate.from_template(template)

final_rag_chain=(
    {
        "context":retrieval_chain,
        "question":itemgetter("question")
    }
    | prompt
    | llm
    | StrOutputParser()
)
final_rag_chain.invoke({"question":question})

'Task decomposition for LLM (Large Language Model) agents involves breaking down complex tasks into smaller and simpler steps. This can be done in three ways:\n\n1. Using simple prompting: The model is instructed with prompts like "Steps for XYZ." or "What are the subgoals for achieving XYZ?"\n2. Using task-specific instructions: For example, "Write a story outline" for writing a novel.\n3. With human inputs.\n\nTask decomposition can also be done using techniques such as Chain of Thought (CoT) and Tree of Thoughts (Yao et al. 2023), which transform big tasks into multiple manageable tasks and shed light on the model\'s thinking process.'